# 文件数据存取

# 一、Pandas Series复习

pandas Series（翻译为系列）是个一维数据结构，而DataFrame的部分操作也非常类似.

In [2]:
import pandas as pd

Series有一个索引和相关的数据。我们可以使用索引来访问数据。我们还了解到，我们可以通过索引和值的隐式位置直接访问它们。如果我们添加隐式位置，一个Series看起来是这样的: 


| Location (Implicit)| Index | Value | 
| - | - | - |
|0| Stones     |    1962
|1| Beatles    |    1960
|2| Zeppelin     |  1968
|3| Pink Floyd |    1965
|4| Pink Floyd |    2012

下面是示例代码:

In [3]:
bands_founded = pd.Series([1962, 1960, 1968, 1965, 2012],
                         name="founded",
                         index=["Stones", "Beatles", "Zeppelin", "Pink Floyd", "Pink Floyd"])
bands_founded

Stones        1962
Beatles       1960
Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2012
Name: founded, dtype: int64

请注意，当我们访问具有多个索引的数据时，我们并没有获得与上述情况相同的简单数据类型，而是获得了另一个Series(系列):

In [4]:
bands_founded["Pink Floyd"]

Pink Floyd    1965
Pink Floyd    2012
Name: founded, dtype: int64

## 索引与切片Indexing，slicing

索引和切片在很大程度上类似于普通的python，但不只是直接使用方括号表示法，建议使用“iloc”按位置进行索引，使用“loc”按索引进行索引. 


In [5]:
# 按位置进行切片
bands_founded.iloc[1:3]

Beatles     1960
Zeppelin    1968
Name: founded, dtype: int64

In [6]:
# 按照索引名称进行索引- 注意包括最后一个元素
bands_founded.loc["Zeppelin" : "Pink Floyd"]

Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2012
Name: founded, dtype: int64

Series可以用数组进行索引，这在普通python中是不可能的:

In [6]:
bands_founded.loc[["Beatles", "Pink Floyd"]]

Beatles       1960
Pink Floyd    1965
Pink Floyd    2012
Name: founded, dtype: int64

## Masking（称为掩码或者屏蔽） Series

通过pandas，我们可以创建布尔数组，用于屏蔽和过滤数据集。在下面的表达式中，我们将创建一个新数组，该数组将使用一种名为**广播Broadcasting**的技术为1964年后形成的每个乐队生成“True”值

In [8]:
mask = bands_founded > 1964
mask

Stones        False
Beatles       False
Zeppelin       True
Pink Floyd     True
Pink Floyd     True
Name: founded, dtype: bool

我们能使用boolean掩码（屏蔽码）来过滤series:

In [9]:
# 应用掩码到原始数组
bands_founded[mask]

Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2012
Name: founded, dtype: int64

简洁形式会是：

In [10]:
bands_founded[bands_founded > 1965]

Zeppelin      1968
Pink Floyd    2012
Name: founded, dtype: int64

## 探索系列Series

在一个系列Series中探索数据有多种方法。我们可以得到统计属性的概述，或者计算单个属性:

In [11]:
#由列表生成Series，求均值和描述性统计
numbers = pd.Series([1962, 1960, 1968, 1965, 2012, 2016])
numbers.mean()

1980.5

In [11]:
numbers.describe()

count       6.000000
mean     1980.500000
std        26.120873
min      1960.000000
25%      1962.750000
50%      1966.500000
75%      2001.000000
max      2016.000000
dtype: float64

## 排序Sorting

In [13]:
sorted_numbers = numbers.sort_values(ascending=False)
sorted_numbers

5    2016
4    2012
2    1968
3    1965
0    1962
1    1960
dtype: int64

## 重置索引--按照实际位置重新编号

In [14]:
sorted_numbers.reset_index(drop=True)

0    2016
1    2012
2    1968
3    1965
4    1962
5    1960
dtype: int64

## 映射Mapping - 应用函数

映射Mapping应用函数到Series中的每个元素: 

In [15]:
import datetime

# 将一个整数年转换为一个日期，假设1月1日是日期和月份.
def to_date(year):
    return datetime.date(int(year), 1, 1)
    
sorted_numbers.map(to_date)

5    2016-01-01
4    2012-01-01
2    1968-01-01
3    1965-01-01
0    1962-01-01
1    1960-01-01
dtype: object

# 二. 读写文本文件数据

到目前为止，我们主要使用直接在代码中指定的数据。当然，这不是特别可扩展的。我们希望从文件中加载数据，并最终连接到数据库和api。
数据通常以结构化文件格式存储，如CSV、JSON或XML。我们将在这个类中遇到所有这些文件格式。最简单的(也是结构最少的)是CSV -逗号分隔值-文件。CSV不是一种正式的文件格式，而是一个以文本文件表示的表，其中单元格由分隔符分隔。通常，第一行表示标题。分隔符可以是制表符、分号、冒号等:
```
Artist, Album, Genre
Michael Jackson, Bad, Pop, funk, rock
``` 
在这里，Album是由逗号分隔的多种Genre。但是，逗号也用于分隔各个列。为了解决这个问题，通常使用双引号来表示引号中包含的所有元素都不是定界符:

```
Artist, Album, Genre
Michael Jackson, Bad, "Pop, funk, rock"
``` 

现在，可以清楚`Pop, funk, rock`应该属于一个单元. 

我们准备了数据集供练[hit_albums.csv]. 数据看起来如下所示：

```
Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)
Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65
AC/DC,Back in Black,1980,Hard rock,25.9,50
Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45
Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44
...
```

读取CSV文件有多种方法。我们将首先介绍Python的基本读(和写)操作，但很快将转到Python和pandas中用于CSV文件的特定解析器。. 

## 1. 基本文件操作

要读取一个文件，我们首先必须指定文件路径，并指定我们是否想要读取(r), 写(w),既读又写(r+),或追加(a). 

In [2]:
"""
python3中的默认的编码解码方式为utf-8
windows默认使用的编码方式为gbk
当读取一个windows文件(gbk),可以指定读取时使用的编码encoding=('gbk')或者不需要指定
with open('a.txt','r','encoding=(gbk)') as f:这样就能读取出a.txt文件的字符串数据
当python写入文件时,默认使用的utf-8编码,在win上打开会乱码,因为win默认编码是gbk

"""
#Python内置open函数打开文件
#albums_file = open('hit_albums.csv', 'r')
albums_file = open('hit_albums.csv', 'r',encoding=('utf-8'))

我们可以一次读取整个文件。注意，行以特殊字符、换行符或换行符结束`\n`.

In [3]:
#通过文件对象的read函数一次性读取所有文件内容
content = albums_file.read()
content

'Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)\nMichael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65\nAC/DC,Back in Black,1980,Hard rock,25.9,50\nPink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45\nWhitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44\nMeat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43\nEagles,Their Greatest Hits (1971–1975),1976,"Rock, soft rock, folk rock",32.2,42\nBee Gees / Various artists,Saturday Night Fever,1977,Disco,19,40\nFleetwood Mac,Rumours,1977,Soft rock,27.9,40\nShania Twain,Come On Over,1997,"Country, pop",29.6,39\nLed Zeppelin,Led Zeppelin IV,1971,"Hard rock, heavy metal",29,37\nMichael Jackson,Bad,1987,"Pop, funk, rock",20.3,34\nAlanis Morissette,Jagged Little Pill,1995,Alternative rock,24.8,33\nCeline Dion,Falling into You,1996,"Pop, Soft rock",20.2,32\nThe Beatles,Sgt. Pepper\'s Lonely Hearts Club Band,1967,Rock,13.1,32\nEagles,Hotel California

如果打印出来, `\n`就会被翻译为新行: 

In [4]:
print(content)

Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)
Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65
AC/DC,Back in Black,1980,Hard rock,25.9,50
Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45
Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44
Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43
Eagles,Their Greatest Hits (1971–1975),1976,"Rock, soft rock, folk rock",32.2,42
Bee Gees / Various artists,Saturday Night Fever,1977,Disco,19,40
Fleetwood Mac,Rumours,1977,Soft rock,27.9,40
Shania Twain,Come On Over,1997,"Country, pop",29.6,39
Led Zeppelin,Led Zeppelin IV,1971,"Hard rock, heavy metal",29,37
Michael Jackson,Bad,1987,"Pop, funk, rock",20.3,34
Alanis Morissette,Jagged Little Pill,1995,Alternative rock,24.8,33
Celine Dion,Falling into You,1996,"Pop, Soft rock",20.2,32
The Beatles,Sgt. Pepper's Lonely Hearts Club Band,1967,Rock,13.1,32
Eagles,Hotel California,1976,"Rock, soft

在读取文件之后，我们必须再次手动关闭它以释放操作系统资源:

In [5]:
albums_file.close()

或者，我们可以分别读取每一行:

In [6]:
albums_file = open('hit_albums.csv', 'r',encoding='utf-8')
line1 = albums_file.readline();
print(line1)

Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)



我们现在能用[`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) 基于逗号拆分字符串，来创建一个简单的CSV解析器:

In [7]:
#字符串的split函数，可以指定分隔符拆分字符串
line1.split(",")

['Artist',
 'Album',
 'Released',
 'Genre',
 '"Certified sales (millions)"',
 'Claimed sales (millions)\n']

我们可以循环读取文件内容到数组中:

In [8]:
data = []
for line in albums_file:
    data.append(line.split(","))
    
# let's not forget to close the file:
albums_file.close()
data

[['Michael Jackson',
  'Thriller',
  '1982',
  '"Pop',
  ' rock',
  ' R&B"',
  '45.4',
  '65\n'],
 ['AC/DC', 'Back in Black', '1980', 'Hard rock', '25.9', '50\n'],
 ['Pink Floyd',
  'The Dark Side of the Moon',
  '1973',
  'Progressive rock',
  '22.7',
  '45\n'],
 ['Whitney Houston / Various artists',
  'The Bodyguard',
  '1992',
  '"Soundtrack/R&B',
  ' soul',
  ' pop"',
  '27.4',
  '44\n'],
 ['Meat Loaf',
  'Bat Out of Hell',
  '1977',
  '"Hard rock',
  ' progressive rock"',
  '20.6',
  '43\n'],
 ['Eagles',
  'Their Greatest Hits (1971–1975)',
  '1976',
  '"Rock',
  ' soft rock',
  ' folk rock"',
  '32.2',
  '42\n'],
 ['Bee Gees / Various artists',
  'Saturday Night Fever',
  '1977',
  'Disco',
  '19',
  '40\n'],
 ['Fleetwood Mac', 'Rumours', '1977', 'Soft rock', '27.9', '40\n'],
 ['Shania Twain', 'Come On Over', '1997', '"Country', ' pop"', '29.6', '39\n'],
 ['Led Zeppelin',
  'Led Zeppelin IV',
  '1971',
  '"Hard rock',
  ' heavy metal"',
  '29',
  '37\n'],
 ['Michael Jackson', 'Ba

In [9]:
#读取单个行与单元
data[0][1]

'Thriller'

In [34]:
data[0]

['Michael Jackson',
 'Thriller',
 '1982',
 '"Pop',
 ' rock',
 ' R&B"',
 '45.4',
 '65\n']

正如我们所看到的，这并没有恰当地处理我们对"Pop, rock, R&B"的双引号式转义。此外，数字仍然被视为字符串，换行符也被附加到最后一个单元格。
我们当然可以改进解析器来处理这些问题，但幸运的是，现有的解析CSV文件的方法使这一点变得更容易.

### 写入

我们可以使用"w"标识打开文件写入操作g.这里我们也使用[`with`](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) 关键字, 即使出现问题，它也会为我们关闭文件:

In [10]:
with open('my_file.txt', 'w') as new_file:
    new_file.write("Hello World\nAre you still spinning?\n")

## 练习 1: 读写数据

文件 [grades.csv](grades.csv) 是有名字和字母级别的文件:

```
Alice; A
Bob; B
Robert; A
Richard; C
```

读文件到数组，添加一个GPA到学生的行(A=4,B=3,C=2,D=1). 

提示: 函数 [strip()](https://docs.python.org/3/library/stdtypes.html#str.strip) 从字符串中删除尾随的空白或换行.

写这个文件到新文件`grades_gpa.csv`

In [13]:
gpas = {"A":4, "B":3, "C":2, "D":1}

data = []
# 在这里继续，补充后续操作代码

## 2. 使用CSV库读写CSV文件

参考：https://baike.baidu.com/item/CSV/10739?fr=aladdin

逗号分隔值（Comma-Separated Values，CSV，有时也称为字符分隔值，因为分隔字符也可以不是逗号），其文件以纯文本形式存储表格数据（数字和文本）。纯文本意味着该文件是一个字符序列，不含必须像二进制数字那样被解读的数据。CSV文件由任意数目的记录组成，记录间以某种换行符分隔；每条记录由字段组成，字段间的分隔符是其它字符或字符串，最常见的是逗号或制表符。通常，所有记录都有完全相同的字段序列。

“CSV”并不是一种单一的、定义明确的格式（尽管RFC 4180有一个被通常使用的定义）。因此在实践中，术语“CSV”泛指具有以下特征的任何文件：
    纯文本，使用某个字符集，比如ASCII、Unicode、EBCDIC或GB2312；
    由记录组成（典型的是每行一条记录）；
    每条记录被分隔符分隔为字段（典型分隔符有逗号、分号或制表符；有时分隔符可以包括可选的空格）；
    每条记录都有同样的字段序列。

### 规则
1 开头是不留空，以行为单位。
2 可含或不含列名，含列名则居文件第一行。
3 一行数据不跨行，无空行。
4 以半角逗号（即，）作分隔符，列为空也要表达其存在。
5 列内容如存在半角引号（即"），替换成半角双引号（""）转义，即用半角引号（即""）将该字段值包含起来。
6 文件读写时引号，逗号操作规则互逆。
7 内码格式不限，可为 ASCII、Unicode 或者其他。
8 不支持数字
9 不支持特殊字符

我们可以使用CSV库来帮助读取数据。它接受一个“delimiter”和一个“quotechar”参数，后者对我们的双引号非常有用:

In [14]:
# 导入csv库
import csv

# 初始化顶级数组
data_values = []

# 打开文件并把行数据作为数组追加到data_values中.如果出现乱码，需要在open函数中设置encoding参数
with open('hit_albums.csv',encoding="utf-8") as csvfile:
    # 请注意，通常我们可以互换使用'和"
    # 对于quotechar，我们使用'以便我们可以使用"而不用转义
    filereader = csv.reader(csvfile, delimiter=',', quotechar='"')
    # 这里的行是个数组
    for row in filereader:
        print("Row: " + str(row))
        data_values.append(row)

# 保存头数据（header，对应列名）到单独的数组中
header = data_values.pop(0)
   
print()    
print(header)
print()
print(data_values)

Row: ['Artist', 'Album', 'Released', 'Genre', 'Certified sales (millions)', 'Claimed sales (millions)']
Row: ['Michael Jackson', 'Thriller', '1982', 'Pop, rock, R&B', '45.4', '65']
Row: ['AC/DC', 'Back in Black', '1980', 'Hard rock', '25.9', '50']
Row: ['Pink Floyd', 'The Dark Side of the Moon', '1973', 'Progressive rock', '22.7', '45']
Row: ['Whitney Houston / Various artists', 'The Bodyguard', '1992', 'Soundtrack/R&B, soul, pop', '27.4', '44']
Row: ['Meat Loaf', 'Bat Out of Hell', '1977', 'Hard rock, progressive rock', '20.6', '43']
Row: ['Eagles', 'Their Greatest Hits (1971–1975)', '1976', 'Rock, soft rock, folk rock', '32.2', '42']
Row: ['Bee Gees / Various artists', 'Saturday Night Fever', '1977', 'Disco', '19', '40']
Row: ['Fleetwood Mac', 'Rumours', '1977', 'Soft rock', '27.9', '40']
Row: ['Shania Twain', 'Come On Over', '1997', 'Country, pop', '29.6', '39']
Row: ['Led Zeppelin', 'Led Zeppelin IV', '1971', 'Hard rock, heavy metal', '29', '37']
Row: ['Michael Jackson', 'Bad', '19

要对这个表的数值维进行计算，我们还需要将字符串转换为数字。这里最后一列`Claimed sales (millions)`不是每一行都有值，因此转换会抛出`ValueError` 异常. [Exceptions](https://docs.python.org/3/reference/compound_stmts.html#try)是能够捕获并抛出的错误状态:

In [15]:
for row in data_values: 
    row[2] = int(row[2])
    row[4] = float(row[4])
    # 需要尝试捕获异常，因为该列包含NaN值
    try:
        row[5] = float(row[5])
    except ValueError: 
        row[5] = None
    
data_values

[['Michael Jackson', 'Thriller', 1982, 'Pop, rock, R&B', 45.4, 65.0],
 ['AC/DC', 'Back in Black', 1980, 'Hard rock', 25.9, 50.0],
 ['Pink Floyd',
  'The Dark Side of the Moon',
  1973,
  'Progressive rock',
  22.7,
  45.0],
 ['Whitney Houston / Various artists',
  'The Bodyguard',
  1992,
  'Soundtrack/R&B, soul, pop',
  27.4,
  44.0],
 ['Meat Loaf',
  'Bat Out of Hell',
  1977,
  'Hard rock, progressive rock',
  20.6,
  43.0],
 ['Eagles',
  'Their Greatest Hits (1971–1975)',
  1976,
  'Rock, soft rock, folk rock',
  32.2,
  42.0],
 ['Bee Gees / Various artists',
  'Saturday Night Fever',
  1977,
  'Disco',
  19.0,
  40.0],
 ['Fleetwood Mac', 'Rumours', 1977, 'Soft rock', 27.9, 40.0],
 ['Shania Twain', 'Come On Over', 1997, 'Country, pop', 29.6, 39.0],
 ['Led Zeppelin',
  'Led Zeppelin IV',
  1971,
  'Hard rock, heavy metal',
  29.0,
  37.0],
 ['Michael Jackson', 'Bad', 1987, 'Pop, funk, rock', 20.3, 34.0],
 ['Alanis Morissette',
  'Jagged Little Pill',
  1995,
  'Alternative rock',
  

所以，这里我们有一个矩阵，我们可以利用它。实际上，我们可能想要以稍微不同的方式构造数据—我们不想把每一行当作一个数组，我们想要把每个维(列)当作一个数组，因为这样可以使列齐次，并且便于计算平均值，等等. 

## 使用CSV库写入CSV文件

In [35]:
import csv
with open('test.csv', 'w') as f:
    writer = csv.writer(f)
    # 写入表头，表头是单行数据
    writer.writerow(['name', 'age', 'sex'])
    data = [
        ('huangyuan', 20, 'male'),
        ('zhanglan', 22, 'female')
    ]
    # 写入这些多行数据
    writer.writerows(data)

## 3. 使用Pandas读写CSV

In [18]:
#Pandas的read_csv函数。Pandas从数据源读取数据的函数，基本都是以read_开头
import pandas as pd
hit_albums = pd.read_csv("hit_albums.csv")
hit_albums.head()   #缺省查看头5行

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65.0
1,AC/DC,Back in Black,1980,Hard rock,25.9,50.0
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45.0
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44.0
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43.0


In [38]:
#还可以使用read_table函数，读取空格或分隔符分隔的数据，效果与read_csv函数一样
hit_albums_table=pd.read_table('hit_albums.csv', sep=',')
hit_albums_table.head()

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65.0
1,AC/DC,Back in Black,1980,Hard rock,25.9,50.0
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45.0
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44.0
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43.0


Pandas提供了一种异常强大的方法['read_csv()'](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)  - 也可以参考[这里](http://pandas.pydata.org/pandas-docs/stable/io.html) 获取更多的I/O操作. 

您可以向该方法传递许多参数，如delimiter分隔符、quote-chars引号字符等，但在我们的示例中，默认参数是有效的。
我们还刚刚创建了我们的第一个数据框!接下来让我们详细了解一下数据框。 

In [50]:
#反之，还可以使用to_开头的函数，将文件DataFrame数据写入文件
hit_albums.to_csv("dfout-test1.csv")   #带有索引
hit_albums.to_csv("dfout-test2.csv",index=None)   #不带索引
hit_albums.to_csv("dfout-test3.csv",index=None,header=False)  #不带索引,不带列名头

## 如果读取文件太大，需要考虑分块读取及显示部分那内容
如：
pd.options.display.max_rows = 10
read_csv函数设置一次读取数量的参数，如nrows限定读取行数；chunksize分批次处理文件，每次处理多少数据

In [48]:
pd.options.display.max_rows = 20
hit_albums = pd.read_csv("hit_albums.csv",nrows=6)
print(hit_albums)
#有chunksize，返回的就是TextFileReader对象,需要循环处理每批次返回的数据
hit_albums_table=pd.read_table('hit_albums.csv', chunksize=3) 
#hit_albums_table=pd.read_table('hit_albums.csv', sep=',',chunksize=3) 
for index,data in enumerate(hit_albums_table):
    if index<5:
        print(data)

                              Artist                            Album  \
0                    Michael Jackson                         Thriller   
1                              AC/DC                    Back in Black   
2                         Pink Floyd        The Dark Side of the Moon   
3  Whitney Houston / Various artists                    The Bodyguard   
4                          Meat Loaf                  Bat Out of Hell   
5                             Eagles  Their Greatest Hits (1971–1975)   

   Released                        Genre  Certified sales (millions)  \
0      1982               Pop, rock, R&B                        45.4   
1      1980                    Hard rock                        25.9   
2      1973             Progressive rock                        22.7   
3      1992    Soundtrack/R&B, soul, pop                        27.4   
4      1977  Hard rock, progressive rock                        20.6   
5      1976   Rock, soft rock, folk rock                

# 4. 读写JSON数据

JSON(JavaScript Object Notation) 是一种轻量级的数据交换格式。JSON采用完全独立于语言的文本格式，这些特性使JSON成为理想的数据交换语言。易于人阅读和编写，同时也易于机器解析和生成。
1.基础结构
JSON建构于两种结构：

“名称/值”对的集合（A collection of name/value pairs）。不同的语言中，它被理解为对象（object），记录（record），结构（struct），字典（dictionary），哈希表（hash table），有键列表（keyed list），或者关联数组 （associative array）。
值的有序列表（An ordered list of values）。在大部分语言中，它被理解为数组（array）。

2.基础示例
简单地说，JSON 可以将 JavaScript 对象中表示的一组数据转换为字符串，然后就可以在函数之间轻松地传递这个字符串，或者在异步应用程序中将字符串从 Web 客户机传递给服务器端程序。这个字符串看起来有点儿古怪，但是 JavaScript 很容易解释它，而且 JSON 可以表示比"名称 / 值对"更复杂的结构。例如，可以表示数组和复杂的对象，而不仅仅是键和值的简单列表。
表示名称 / 值对
按照最简单的形式，可以用下面这样的 JSON 表示 "名称 / 值对" ：{ "firstName": "Brett" }

这个示例非常基本，而且实际上比等效的纯文本 "名称 / 值对" 占用更多的空间：firstName=Brett

但是，当将多个"名称 / 值对"串在一起时，JSON 就会体现出它的价值了。首先，可以创建包含多个"名称 / 值对"的 记录，比如：
{ "firstName": "Brett", "lastName":"McLaughlin", "email": "aaaa" }
从语法方面来看，这与"名称 / 值对"相比并没有很大的优势，但是在这种情况下 JSON 更容易使用，而且可读性更好。例如，它明确地表示以上三个值都是同一记录的一部分；花括号使这些值有了某种联系。
表示数组
当需要表示一组值时，JSON 不但能够提高可读性，而且可以减少复杂性。例如，假设您希望表示一个人名列表。在 XML 中，需要许多开始标记和结束标记；如果使用典型的 名称 / 值 对（就像在本系列前面文章中看到的那种名称 / 值对），那么必须建立一种专有的数据格式，或者将键名称修改为 person1-firstName这样的形式。

如果使用 JSON，就只需将多个带花括号的记录分组在一起：

{ "people": [

    { "firstName": "Brett", "lastName":"McLaughlin", "email": "aaaa" },

    { "firstName": "Jason", "lastName":"Hunter", "email": "bbbb"},

    { "firstName": "Elliotte", "lastName":"Harold", "email": "cccc" }

    ]}
可以使用相同的语法表示多个值（每个值包含多个记录）,在不同的主条目之间，记录中实际的名称 / 值对可以不一样。JSON 是完全动态的，允许在 JSON 结构的中间改变表示数据的方式。例如：

{ "programmers": [

    { "firstName": "Brett", "lastName":"McLaughlin", "email": "aaaa" },

    { "firstName": "Jason", "lastName":"Hunter", "email": "bbbb" },

    { "firstName": "Elliotte", "lastName":"Harold", "email": "cccc" }

    ],

    "authors": [

    { "firstName": "Isaac", "lastName": "Asimov", "genre": "science fiction" },

    { "firstName": "Tad", "lastName": "Williams", "genre": "fantasy" },

    { "firstName": "Frank", "lastName": "Peretti", "genre": "christian fiction" }

    ],

    "musicians": [

    { "firstName": "Eric", "lastName": "Clapton", "instrument": "guitar" },

    { "firstName": "Sergei", "lastName": "Rachmaninoff", "instrument": "piano" }

    ] }
    

使用json模块操作json数据

In [51]:
#注意观察JSON格式：JavaScriptObjectNotation，可嵌套的名称/值对序列
#对于较长的字符串，需要使用""""""
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [52]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [59]:
#将json对象序列化
asjson = json.dumps(result)
print(asjson)

{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}


使用Pandas操作JSON数据

In [64]:
import pandas as pd
data = pd.read_json("Sample.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [65]:
#将DataFrame数据输出为json格式
print(data.to_json())  #索引形式
print("记录形式：",data.to_json(orient='records'))  #记录形式

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


# 5. 读写XML 与 HTML
一.XML介绍-参考：https://blog.csdn.net/vipers_/article/details/89679536

1、概述：

    xml:即可扩展标记语言，xml是互联网数据传输的重要工具，它可以跨越互联网任何的平台，不受编程语言和操作系统的限制，可以说它是一个拥有互联网最高级别通行证的数据携带者。xml是当前处理结构化文档信息中相当给力的技术，xml有助于在服务器之间穿梭结构化数据，这使得开发人员更加得心应手的控制数据的存储和传输。

Xml用于标记电子文件使其具有结构性的标记语言，可以用来标记数据、定义数据类型，是一种允许用户对自己的标记语言进行定义的源语言。Xml是标准通用标记语言（SGML）的子集，非常适合Web传输。XML提供统一的方法来描述和交换独立于应用程序或供应商的结构化数据。

2、xml的特点及作用:

特点：

v xml与操作系统、编程语言的开发平台都无关；

v 实现不同系统之间的数据交互。

作用：

v 配置应用程序和网站；

v 数据交互；

v Ajax基石。

在配置文件里边所有的配置文件都是以XMl的格式来编写的。

跨平台进行数据交互，它可以跨操作系统，也可以跨编程语言的平台。

Ajax是现在目前比较流行的一个网络交互的技术。Ajax里面最后一个x实际上就是xml的缩写。Xml（Extensible Markup Language）是可扩展标记语言

一个xml文档必须要有第一行的声明和它的文档元素的描述信息就可以了。

3、xml声明

例子：

注：xml声明一般是xml文档的第一行；xml声明由以下几个部分组成：

4、根元素

它是xml文档里面唯一的；它的开始是放在最前面，结束是放在最后面。

5、元素：

(1) 所有的xml元素都必须有结束标签；

(2) xml标签对大小写敏感；

(3)  xml必须正确地嵌套；

（4）元素的命名规则：

名称中可以包含字母、数字或者其他的字符；

名称不能以数字或者标点符号开始；

名称中不能包含空格。

（5）空元素

6、属性

（1）语法

<元素名 属性名=“属性值”/>

例：<Student ID=“S100”>

       <Name>Tom</Name>

</Student>

(2)注意：

属性值用双引号包裹；一个元素可以有多个属性，它的基本格式为：

<元素名 属性名=“属性值” 属性名=“属性值”>;

属性值中不能够直接包含<.”,&。

7、实体：

在xml中，一些字符拥有特殊的意义。如果把字符“<”放在xml元素中，会发生错误，这是因为解析器会把它当作新元素的开始，这样会产生xml错误：

为了避免这个错误，请用实体引用来代替“<”字符：

xml中5个预定义实体

8、注释

注:注释内容中不要出现”--”;不要把注释放在标签中间；注释不能嵌套。

9、总结：

（1）xml描述的是文档的内容与语义，而不是文档应当如何显示；

（2）格式正规（well formed）的xml文档

遵循如下规则的xml文档称为格式正规的xml文档：

v 必须有xml声明语句；

v 必须有且仅有一个根元素；

v 标签大小写敏感；

v 属性值用双引号；

v 标签成对；

v 空标签关闭；

v 元素正确嵌套。

(3)有效的（valid）xml文档。首先xml文档是个格式正规的xml文档，然后又需要满足DTD的要求，这样的xml文档称为有效的xml文档；

10、解析器

11、命名空间

11.1、xml命名空间（xml Namespaces）

（1）xml命名空间提供避免元素命名冲突的方法。

（2）在xml中，元素名称是由开发者定义的，当两个不同的文档使用相同的元素名时，就会发生命名冲突。

11.2、xml命名空间示例

（1）使用前缀示例

与仅仅使用前缀不同，我们为标签添加了一个xmlns属性，这样就为前缀赋予了一个与某个命名空间相关联的限定名称

二. HTML简介

1、HTML基本文档格式—

参考：https://blog.csdn.net/xtyzmnchen/article/details/87917907

2、HTML 标签关系

嵌套关系：父子

并列关系：兄弟姐妹

3、 HTML 标签分类

双标记 <标记名></标记名>

单标记 <标记名/> 

三. Python工具：需要安装lxml模块

conda install lxml

pip install beautifulsoup4 html5lib

In [66]:
#爬虫有关工具不在本章说明，仅介绍pandas对应工具
tables = pd.read_html('fdic_failed_bank_list.html')
len(tables)
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


使用lxml.objectify解析XML

下面是XML格式数据，双击单元即可看到源文件格式

<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

In [73]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
print(parsed)
root = parsed.getroot()  #返回的是文档对象模型DOM数据结构
print(type(root))

<class 'lxml.objectify.ObjectifiedElement'>


In [70]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']
#注意：
for elt in root.INDICATOR:
    el_data = {}  #以字典形式存储每行记录数据
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)   #多行记录以列表形式存储

In [74]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


In [76]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [78]:
print("root:",root)
root.get('href')
root.text

root: Google


'Google'

# 三、二进制及其他文件格式

# 6. 读写二进制数据格式
这里主要了解Python对象的序列化和反序列化.参考：https://www.cnblogs.com/soulgou123/p/9829378.html

1、什么是序列化与反序列化？
#我们把对象（或变量）从内存变成可存储或可传输的过程称之为序列化，在python中被称为pickle；

#自定义的类的实例如何保存在一个文件中？如何从文件中读取数据，并让他们在内存中再次恢复成自己对应的类的实例？

#按照某种规则，把内存中的数据保存到文件中，文件是一个字节序列，所以必须要把内存数据转换成为字节序列，输出到文件，这就是序列化；反之，从文件的字节恢复到内存，就是反序列化；

2、为什么要序列化？
#持久化保持状态；

#跨平台的数据交互；

3、pickle
#pickle模块是python专用的持久化模块，可以持久化包括自定义类在内的各种数据；只能在python程序之间进行数据交换；

#方法：

dumps 　　将对象序列化成bytes对象；
dump 　　 对象序列化到文件对象，就是存入文件；
loads　　  从bytes对象反序列化；
load　　    对象反序列化，从文件读取数据；
#注意：在使用dump()序列化时候，打开文件必须要以wb模式，使用load()反序列化，打开文件必须以rb模式；

In [94]:
#Pandas的DataFrame对象的序列化和反序列化
frame = pd.read_csv("hit_albums.csv", nrows=6)
print(frame)
#DataFrame对象持久化到二进制文件
frame.to_pickle('frame_pickle')

                              Artist                            Album  \
0                    Michael Jackson                         Thriller   
1                              AC/DC                    Back in Black   
2                         Pink Floyd        The Dark Side of the Moon   
3  Whitney Houston / Various artists                    The Bodyguard   
4                          Meat Loaf                  Bat Out of Hell   
5                             Eagles  Their Greatest Hits (1971–1975)   

   Released                        Genre  Certified sales (millions)  \
0      1982               Pop, rock, R&B                        45.4   
1      1980                    Hard rock                        25.9   
2      1973             Progressive rock                        22.7   
3      1992    Soundtrack/R&B, soul, pop                        27.4   
4      1977  Hard rock, progressive rock                        20.6   
5      1976   Rock, soft rock, folk rock                

In [95]:
#从持久化二进制文件读取，返回DataFrame对象
pd.read_pickle('frame_pickle')

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65
1,AC/DC,Back in Black,1980,Hard rock,25.9,50
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43
5,Eagles,Their Greatest Hits (1971–1975),1976,"Rock, soft rock, folk rock",32.2,42


In [97]:
#pickle模块的序列化和反序列化
import pickle
df1=pickle.load(open('frame_pickle',"rb"))
print(df1)
pickle.dump(df1,open('frame_pickle2',"wb"))

                              Artist                            Album  \
0                    Michael Jackson                         Thriller   
1                              AC/DC                    Back in Black   
2                         Pink Floyd        The Dark Side of the Moon   
3  Whitney Houston / Various artists                    The Bodyguard   
4                          Meat Loaf                  Bat Out of Hell   
5                             Eagles  Their Greatest Hits (1971–1975)   

   Released                        Genre  Certified sales (millions)  \
0      1982               Pop, rock, R&B                        45.4   
1      1980                    Hard rock                        25.9   
2      1973             Progressive rock                        22.7   
3      1992    Soundtrack/R&B, soul, pop                        27.4   
4      1977  Hard rock, progressive rock                        20.6   
5      1976   Rock, soft rock, folk rock                

# 7.读写HDF5数据格式
参考：https://blog.csdn.net/u013177494/article/details/72629199

官网：https://www.hdfgroup.org/
## 什么是HDF格式？
1.HDF是包含多种信息的一个单文件，所有的信息放在同一个文件中。

2.在HDF通过特定文件结构来存储多种不同信息。


HDF 是用于存储和分发科学数据的一种自我描述、多对象文件格式。HDF 是由美国国家超级计算应用中心（NCSA）创建的，以满足不同群体的科学家在不同工程项目领域之需要。HDF 可以表示出科学数据存储和分布的许多必要条件。HDF 被设计为：

自述性：对于一个HDF 文件里的每一个数据对象，有关于该数据的综合信息（元数据）。在没有任何外部信息的情况下，HDF 允许应用程序解释HDF文件的结构和内容。

通用性：许多数据类型都可以被嵌入在一个HDF文件里。例如，通过使用合适的HDF 数据结构，符号、数字和图形数据可以同时存储在一个HDF 文件里。

灵活性：HDF允许用户把相关的数据对象组合在一起，放到一个分层结构中，向数据对象添加描述和标签。它还允许用户把科学数据放到多个HDF 文件里。

扩展性：HDF极易容纳将来新增加的数据模式，容易与其他标准格式兼容。

跨平台性：HDF 是一个与平台无关的文件格式。HDF 文件无需任何转换就可以在不同平台上使用。

3.HDF基本数据类型
HDF 提供6 种基本数据类型：光栅图像（Raster Image），调色板（Palette ），科学数据集（Scientific Data Set），注解（Annotation），虚拟数据（Vdata）和虚拟组（Vgroup）。

4.HDF文件格式
最好的办法是把HDF 文件看成为一本多章节书。HDF 文件是“数据书”，其中每章都包含一个不同类型的数据内容。正如书籍用一个目录表列出它的章节一样，HDF文件用“data index”（数据索引）列出其数据内容
HDF 文件结构包括一个file id（文件号）、至少一个 data descriptor （数据描述符）、没有或多个 data element（数据内容）数据内容。

In [99]:
#HDF5即Hierarchy Data Format标准的version5 格式
import pandas as pd
import numpy as np
frame = pd.DataFrame({'a': np.random.randn(100)})
#创建HDF存储对象store
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [101]:
store['obj1'].head()

,a
0,1.026020
1,0.323476
2,0.487712
3,-0.047118
4,0.445724


In [102]:
#HDF存储对象操作与写入
store.put('obj2', frame, format='table')
aa=store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()
aa

,a
10,-0.490838
11,-0.737061
12,-0.654087
13,0.615824
14,0.058159
15,0.898758


In [103]:
#DataFrame对象的hdf格式文件的读写操作
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,1.026020
1,0.323476
2,0.487712
3,-0.047118
4,0.445724


In [104]:
#删除文件。或者调用操作系统命令形式：!del 或rm 命令
import os
os.remove('mydata.h5')

# 8.读写微软Excel文件
Excel格式不再介绍。读写时候需要注意编码格式encoding、表单号sheet、表头head.下面是对应Pandas的函数形式：

1.to_excel(
    excel_writer,
    sheet_name='Sheet1',
    na_rep='',
    float_format=None,
    columns=None,
    header=True,
    index=True,
    index_label=None,
    startrow=0,
    startcol=0,
    engine=None,
    merge_cells=True,
    encoding=None,
    inf_rep='inf',
    verbose=True,
    freeze_panes=None,
)

2.read_excel(
    io,
    sheet_name=0,
    header=0,
    names=None,
    index_col=None,
    usecols=None,
    squeeze=False,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skiprows=None,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    verbose=False,
    parse_dates=False,
    date_parser=None,
    thousands=None,
    comment=None,
    skip_footer=0,
    skipfooter=0,
    convert_float=True,
    mangle_dupe_cols=True,
    **kwds,
)


In [105]:
#一种读取Excel文件方法：生成ExcelFile对象，然后调用read_excel方法
xlsx = pd.ExcelFile('9-1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

,身高,体重
0,160,60
1,162,70
2,162,56
3,163,67
4,164,67
...,...,...
21,181,89
22,182,75
23,183,81
24,184,82


In [106]:
#直接将文件作为参数读取，如果有乱码或者指定表头所在的行号，可设置参数encoding,header
df2 = pd.read_excel('9-1.xlsx', 'Sheet1')
df2.head()

,身高,体重
0,160,60
1,162,70
2,162,56
3,163,67
4,164,67


In [126]:
#写入Excel文件，使用ExcelWriter作为文件操作对象
writer = pd.ExcelWriter('9-1-bak.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()
writer.close()

In [116]:
#使用不使用ExcelWriter作为文件操作对象，而直接调用DataFrame的to_excel函数，会覆盖原来的Excel文件，原有文件内容消失
frame.to_excel('9-1-bak.xlsx','Sheet2')

In [127]:
#下面语句会在原来的Excel文件基础上，添加一个Sheet.注意mode : {'w', 'a'}, 缺省是'w'即重写，'a'指追加
writer = pd.ExcelWriter('9-1-bak.xlsx',mode={'a'})
#注意：这种方式可以在打开的Excel文件连续追加，但原始内容丢失
frame.to_excel(excel_writer=writer,sheet_name='Sheet2')
df2.to_excel(excel_writer=writer,sheet_name='Sheet3')
writer.save()
writer.close()
"""
假如想要对已经存在的excel文件进行修改，可以使用开源工具包（anaconda已附带）openpyxl
import pandas as pd
from openpyxl import load_workbook
 
writer = pd.ExcelWriter('test.xlsx',engin='openpyxl')
book = load_workbook(writer.path)
writer.book = book
dataframe.to_excel(excel_writer=writer,sheet_name="info5")
writer.save()
writer.close()
"""

'\n假如想要对已经存在的excel文件进行修改，可以使用开源工具包（anaconda已附带）openpyxl\nimport pandas as pd\nfrom openpyxl import load_workbook\n \nwriter = pd.ExcelWriter(\'test.xlsx\',engin=\'openpyxl\')\nbook = load_workbook(writer.path)\nwriter.book = book\ndataframe.to_excel(excel_writer=writer,sheet_name="info5")\nwriter.save()\nwriter.close()\n'

In [131]:
pd.read_excel?


# 9. 数据框Data Frames再复习

数据框是面向列的数据结构，每列是个Pandas的系列Series.

我们已经从文件中加载了一个数据框，但是为了完整起见，让我们在代码中创建一个: 

In [19]:
#从字典创建DataFrame，字典的key作为column名称，values作为列的数据
bandInfo = pd.DataFrame({
        "Name":["Led Zeppelin", "The Beatles", "Rolling Stones", "Radiohead"],
        "No Members":[4, 4, 4, 5],
        "No Albums":[9, 12, 29 ,9]
    })
bandInfo


,Name,No Members,No Albums
0,Led Zeppelin,4,9
1,The Beatles,4,12
2,Rolling Stones,4,29
3,Radiohead,5,9


就像一个系列series，一个数据框有一个索引index，它对应于这里的第一列。在本例中，索引是自动生成的，但是对于这个系列，我们可以为索引使用显式的值。
我们可以访问数据框中的列，它返回一个系列series:

In [20]:
bandInfo["Name"]

0      Led Zeppelin
1       The Beatles
2    Rolling Stones
3         Radiohead
Name: Name, dtype: object

In [21]:
type(bandInfo["Name"])

pandas.core.series.Series

显然，我们可以对这个column/series做所有我们学过的事情. 

前面的示例使用列创建数据框架。我们还可以从行创建一个数据帧。在本例中，这没有太大的意义，但是您可以找到来自数据源的数据，比如CSV文件，按照这个顺序. 

In [22]:
bandInfo2 = pd.DataFrame([
        {"Name":"Led Zeppelin", "No Albums":9, "No Members":4},
        {"Name":"The Beatles", "No Albums":12, "No Members":4},
        {"Name":"Rolling Stones", "No Albums":29, "No Members":4},
        {"Name":"Radiohead", "No Albums":9, "No Members":5},
    ])
bandInfo2

,Name,No Albums,No Members
0,Led Zeppelin,9,4
1,The Beatles,12,4
2,Rolling Stones,29,4
3,Radiohead,9,5


虽然一个系列只有一个轴，但是dataframe有两个轴，一个用于行(index或'0'轴)，一个用于列(colun或'1'轴)。我们可以看看这些轴:

In [23]:
#查看轴信息，返回的分别是RangeIndex和Index对象
bandInfo.axes

[RangeIndex(start=0, stop=4, step=1),
 Index(['Name', 'No Members', 'No Albums'], dtype='object')]

In [24]:
# 通过返回的列表对象直接访问行轴和列轴
bandInfo.axes[0]  #行轴

RangeIndex(start=0, stop=4, step=1)

In [25]:
# 列轴
bandInfo.axes[1]

Index(['Name', 'No Members', 'No Albums'], dtype='object')

## 探索Data Frames

您可能已经注意到，数据框是在Jupyter笔记本中的漂亮HTML表中呈现的。对于小的数据框，只显示所有的数据是有意义的，但是对于大的数据集，比如我们的“hit_albums”数据集，绘制70多行数据可能是令人讨厌的，而对于有数百或数千行的数据集可能是禁止的。默认情况下，数据框只打印有限数量的元素(请注意'…'上面' hit_albums '输出的第30行中，只打印前30行和后30行。
在处理数据时，例如，在转换或加载数据集时，查看原始数据是很重要的，例如，检查转换是否正确完成。然而，通常只需要查看数据的一部分就足够了，例如，前几行和/或最后几行。我们可以使用'head()'和'tail()'函数做到:


In [26]:
# 缺省显示倒数5行，参数也可以指定行数
hit_albums.tail()

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
72,Tracy Chapman,Tracy Chapman,1988,Folk rock,20.0,NaN
73,Usher,Confessions,2004,R&B,20.0,NaN
74,Various artists,Flashdance: Original Soundtrack from the Motio...,1983,Soundtrack,20.0,NaN
75,Whitney Houston,Whitney,1987,"Pop, R&B",20.0,NaN
76,Shakira,Laundry Service,2001,"Pop, Rock",20.0,NaN


In [38]:
#检查数据框的维度，返回的是个元组
hit_albums.shape

(77, 6)

在这里，我们了解到我们的数据集有77行和6列。
我们还可以使用info方法获得关于数据集的更多信息，这对于查看列的数据类型特别有帮助:

In [39]:
hit_albums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
Artist                        77 non-null object
Album                         77 non-null object
Released                      77 non-null int64
Genre                         77 non-null object
Certified sales (millions)    77 non-null float64
Claimed sales (millions)      31 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.7+ KB


对于系列，我们可以得到数据集数值的粗略描述.

In [27]:
#有关数值型列的描述性统计
hit_albums.describe()

,Released,Certified sales (millions),Claimed sales (millions)
count,77.000000,77.000000,31.000000
mean,1988.662338,22.409091,35.354839
std,9.949359,4.521620,7.855140
min,1967.000000,13.100000,30.000000
25%,1983.000000,20.000000,30.000000
50%,1987.000000,20.600000,32.000000
75%,1996.000000,25.000000,39.500000
max,2015.000000,45.400000,65.000000


我们没有看到任何非数值类型的列的描述。但是，我们可以通过直接访问列来获得摘要:

In [28]:
hit_albums["Artist"].describe()

count                  77
unique                 56
top       Michael Jackson
freq                    5
Name: Artist, dtype: object

在这里，我们可以看到Michael Jackson（迈克尔·杰克逊）是名单上排名第一的艺术家，有五张专辑albums。名单中还有其他拥有多张专辑的艺人吗?我们可以使用value_counts()方法来回答这个问题:

In [29]:
hit_albums["Artist"].value_counts()

Michael Jackson                      5
The Beatles                          3
Shania Twain                         3
Various artists                      3
Madonna                              3
Bon Jovi                             2
Britney Spears                       2
Pink Floyd                           2
Adele                                2
Mariah Carey                         2
Celine Dion                          2
Eagles                               2
Backstreet Boys                      2
Whitney Houston                      2
Phil Collins                         1
ABBA                                 1
Lionel Richie                        1
James Horner                         1
TLC                                  1
Alanis Morissette                    1
Nirvana                              1
Cyndi Lauper                         1
Led Zeppelin                         1
Carole King                          1
Eric Clapton                         1
George Michael           

我们可以使用[`corr()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html)方法查看数据框架中的数字列是否相关。默认情况下，这会计算列之间的Pearson相关性，不包括NaN值。不足为奇的是，我们发现certified sales和claimed sales之间有很强的相关性(0.81). 

In [30]:
hit_albums.corr()

,Released,Certified sales (millions),Claimed sales (millions)
Released,1.000000,-0.052109,-0.320593
Certified sales (millions),-0.052109,1.000000,0.810280
Claimed sales (millions),-0.320593,0.810280,1.000000


In [31]:
#可以转置矩阵，行列转换
hit_albums.T

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
Artist,Michael Jackson,AC/DC,Pink Floyd,Whitney Houston / Various artists,Meat Loaf,Eagles,Bee Gees / Various artists,Fleetwood Mac,Shania Twain,Led Zeppelin,...,Prince & the Revolution,Shania Twain,Shania Twain,Supertramp,Tina Turner,Tracy Chapman,Usher,Various artists,Whitney Houston,Shakira
Album,Thriller,Back in Black,The Dark Side of the Moon,The Bodyguard,Bat Out of Hell,Their Greatest Hits (1971–1975),Saturday Night Fever,Rumours,Come On Over,Led Zeppelin IV,...,Purple Rain,The Woman in Me,Up!,Breakfast in America,Private Dancer,Tracy Chapman,Confessions,Flashdance: Original Soundtrack from the Motio...,Whitney,Laundry Service
Released,1982,1980,1973,1992,1977,1976,1977,1977,1997,1971,...,1984,1995,2002,1979,1984,1988,2004,1983,1987,2001
Genre,"Pop, rock, R&B",Hard rock,Progressive rock,"Soundtrack/R&B, soul, pop","Hard rock, progressive rock","Rock, soft rock, folk rock",Disco,Soft rock,"Country, pop","Hard rock, heavy metal",...,"Pop, rock, R&B","Country, pop","Country, pop, world music","Progressive rock, art rock","Pop, rock, R&B",Folk rock,R&B,Soundtrack,"Pop, R&B","Pop, Rock"
Certified sales (millions),45.4,25.9,22.7,27.4,20.6,32.2,19,27.9,29.6,29,...,20,20,20,20,20,20,20,20,20,20
Claimed sales (millions),65,50,45,44,43,42,40,40,39,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 分组Data Frames
通常，我们希望聚合数据。例如，考虑到热门专辑（hit-albums）数据集，我们可能想要询问该列表中每个艺术家总共卖出了多少张专辑。我们可以使用group-by方法来进行这些聚合。

我们可以指定一个列来分组，例如，“Artist”。我们可以查看创建的组:

In [32]:
grouped = hit_albums.groupby("Artist")
grouped.groups

{'ABBA': Int64Index([31], dtype='int64'),
 'AC/DC': Int64Index([1], dtype='int64'),
 'Ace of Base': Int64Index([50], dtype='int64'),
 'Adele': Int64Index([20, 56], dtype='int64'),
 'Alanis Morissette': Int64Index([11], dtype='int64'),
 'Avril Lavigne': Int64Index([57], dtype='int64'),
 'Backstreet Boys': Int64Index([48, 49], dtype='int64'),
 'Bee Gees / Various artists': Int64Index([6], dtype='int64'),
 'Bob Marley & The Wailers': Int64Index([37], dtype='int64'),
 'Bon Jovi': Int64Index([32, 54], dtype='int64'),
 'Boston': Int64Index([58], dtype='int64'),
 'Britney Spears': Int64Index([35, 59], dtype='int64'),
 'Bruce Springsteen': Int64Index([22], dtype='int64'),
 'Carole King': Int64Index([38], dtype='int64'),
 'Celine Dion': Int64Index([12, 18], dtype='int64'),
 'Cyndi Lauper': Int64Index([52], dtype='int64'),
 'Def Leppard': Int64Index([61], dtype='int64'),
 'Dire Straits': Int64Index([23], dtype='int64'),
 'Eagles': Int64Index([5, 14], dtype='int64'),
 'Eminem': Int64Index([55], d

注意，键映射到一组索引。例如，在索引中可以找到Michael Jackson（迈克尔·杰克逊）的专辑 [0, 10, 16, 65, 66].

一旦我们创建了这些组，我们就可以指定如何处理它。一个非常通用的解决方案是' agg() '函数，我们可以传递一个函数来处理数据:

In [33]:
#这里我们将sum函数传递给组，该函数计算列表的和
grouped.agg(sum).head()

,Released,Certified sales (millions),Claimed sales (millions)
Artist,,,
ABBA,1992,29.0,0.0
AC/DC,1980,25.9,50.0
Ace of Base,1993,23.0,0.0
Adele,4026,42.3,30.0
Alanis Morissette,1995,24.8,33.0


We can see here that we've summed up all columns. However, note that NaN plus some number is still NaN. So let's work with a slice of the dataframe instead. 


我们可以看到我们已经把所有列加起来了。但是，注意，NaN加上一些数字仍然是NaN。因此，让我们使用一个dataframe片段来代替。这是一个内置函数定义的聚合，我们仍然创建和，但也乘以一百万。我们使用一个 [lambda 表达式](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) 来定义一个函数:

In [53]:
grouped.agg(lambda rows : sum([cell * 1000000 for cell in rows])).head()

,Certified sales (millions),Claimed sales (millions)
Artist,,
ABBA,29000000.0,NaN
AC/DC,25900000.0,50000000.0
Ace of Base,23000000.0,NaN
Adele,42300000.0,NaN
Alanis Morissette,24800000.0,33000000.0


Lambda表达式只是定义函数的另一种方式，而不需要为其指定名称。它们只对单个语句有效。
这是一个简单的lambda表达式，它返回一个函数，我们将该函数赋值给变量add:

In [54]:
add = lambda a, b : a+b

In [55]:
add(2, 3)

5

Pandas也有两个内置函数来代替通用的agg方法。例如，我们可以调用'sum()':

In [34]:
# 首先，创建数据集的新片段
artists_sales = hit_albums[["Artist","Certified sales (millions)"]]
# 然后根据Artist进行分组，将值求和
aggregated = artists_sales.groupby("Artist").sum()
aggregated.head()

,Certified sales (millions)
Artist,
ABBA,29.0
AC/DC,25.9
Ace of Base,23.0
Adele,42.3
Alanis Morissette,24.8


In [57]:
#排序，获得好看的结果
aggregated.sort_values("Certified sales (millions)", ascending=False).head()

,Certified sales (millions)
Artist,
Michael Jackson,123.3
Shania Twain,69.6
Madonna,69.4
Various artists,65.9
Eagles,53.7
